In [1]:
from itertools import combinations,product
import numpy as np
import random 
import copy 
import pandas as pd 
from enum import Enum


groups = pd.read_excel("groups.xlsx")
rooms = pd.read_excel("rooms.xlsx")
subjects = pd.read_excel("subjects.xlsx")
sessions = pd.read_excel("sessions.xlsx")
teachers = pd.read_excel("teachers.xlsx")
working_day = pd.read_excel("working_day.xlsx")

n_of_groups = 6 

working_days = [1,2,3,4,5]  # "sun", "mon", "tue", "wed", "thu"
max_morning_periods = 3
max_afternoon_periods = 3
max_periods_day = 5

In [2]:
sessions

,session_id,subject_id,session_type
0,s11_1,FMA,lecture
1,s11_2,FMA,lecture
2,s11_3,FMA,lecture
3,s11_4,FMA,tuto
4,s11_5,FMA,tuto
...,...,...,...
103,s32_15,ADB,tuto
104,s32_16,ADB,lab
105,s32_17,GP,lab
106,s32_18,PM,lecture


In [3]:
# session type , teacher , length , student capacity, subject
# room type, capacity
# teacher name , field , priority yes no 
# constraint priority yes no 
# working days 

class Session:
    def __init__(self, session_type, teacher, length, subject,section=1,times=1,group=None):
        self.session_type = session_type
        self.teacher = teacher
        self.length = length
        self.subject = subject
        self.times = 1
        self.group = group # 0 mean iime Series Analysis and Classification lab a section
        self.section = section
        self.room = None #get assinged later in the generation
        
    def __eq__(self, other):
        return (isinstance(other, Session) and
                self.session_type == other.session_type and
                self.teacher == other.teacher and
                self.group == other.group and
                self.section == other.section and
                self.length == other.length and
                self.subject == other.subject)

class Room:
    def __init__(self,id, room_type, capacity):
        self.room_type = room_type
        self.id = id
        self.capacity = capacity
        self.available = [True for _ in range(max_periods_day * len(working_days))]

    def __eq__(self, other):
        return (isinstance(other, Room) and
                self.room_type == other.room_type and
                self.capacity == other.capacity)
    def __hash__(self):
        return hash((self.id))
class Teacher:
    def __init__(self, id, name, field, priority,preferef_period=None,prefered_hours=[],prefered_days=[]):
        self.id = id 
        self.name = name
        self.field = field
        self.priority = priority
        self.prefered_days = prefered_days #must be from 'working_days'
        self.preferef_period = preferef_period # m : morning, f:afternoon, None:anything is fine
        self.available = [True for _ in range(max_periods_day * len(working_days))]
    def __hash__(self):
        return hash((self.id))
    def __eq__(self, other):
        return (isinstance(other, Teacher) and
                self.id == other.id)
class Constraint:
    def __init__(self, priority):
        self.priority = priority

class Period:
    def __init__(self, teacher, session, room, time):
        self.teacher = teacher
        self.session = session
        self.room = room
        self.time = time

    def __eq__(self, other):
        return (isinstance(other, Period) and
                self.teacher == other.teacher and
                self.session == other.session and
                self.room == other.room and
                self.time == other.time)


In [4]:
merged = pd.merge(sessions, subjects, on="subject_id")
merged

,session_id,subject_id,session_type,subject_title,field,subject_category,year,semester
0,s11_1,FMA,lecture,Fundamental Mathematics,Mathematics,Fundamental,1,1
1,s11_2,FMA,lecture,Fundamental Mathematics,Mathematics,Fundamental,1,1
2,s11_3,FMA,lecture,Fundamental Mathematics,Mathematics,Fundamental,1,1
3,s11_4,FMA,tuto,Fundamental Mathematics,Mathematics,Fundamental,1,1
4,s11_5,FMA,tuto,Fundamental Mathematics,Mathematics,Fundamental,1,1
...,...,...,...,...,...,...,...,...
102,s32_15,ADB,tuto,Advanced Databases,Computer Science,Methodologic,3,2
103,s32_16,ADB,lab,Advanced Databases,Computer Science,Methodologic,3,2
104,s32_17,GP,lab,Group-Project,Computer Science,Methodologic,3,2
105,s32_18,PM,lecture,Project Management,Business & Entrepreneurship,Transversal,3,2


In [5]:
### you forgot in the database to include who teaches what ? ( they can teach many things ) 
### and who teaches what group ? 
### how many times do you study that session ?

In [6]:
merged.head()

,session_id,subject_id,session_type,subject_title,field,subject_category,year,semester
0,s11_1,FMA,lecture,Fundamental Mathematics,Mathematics,Fundamental,1,1
1,s11_2,FMA,lecture,Fundamental Mathematics,Mathematics,Fundamental,1,1
2,s11_3,FMA,lecture,Fundamental Mathematics,Mathematics,Fundamental,1,1
3,s11_4,FMA,tuto,Fundamental Mathematics,Mathematics,Fundamental,1,1
4,s11_5,FMA,tuto,Fundamental Mathematics,Mathematics,Fundamental,1,1


In [7]:
rooms.head()

,room_id,room_type,session_type,room_capacity
0,amphi1,Amphi,lecture,120
1,amphi2,Amphi,lecture,120
2,amphi3,Amphi,lecture,120
3,amphi4,Amphi,lecture,120
4,amphi5,Amphi,lecture,120


In [8]:
# read from database and define variables ... ( can change if DB Archi changes )

session_3_year = []
for index, row in merged.iterrows():
    if row['year'] == 3 and row["semester"] == 2:
        session = Session(row['session_type'], "" , 1 ,  row['subject_title'])
        session_3_year.append(session)

available_rooms = []
for index, row in rooms.iterrows():
    room = Room(row['room_id'],row['session_type'], row['room_capacity'])
    available_rooms.append(room)


teachers_3_year = []

groups_teachers = ["Djiroun", "Brairi", "Lakehal", "Hadje Ameur", "Brahimi", "Guessoum", 
                   "Boutorh", "Lasla", "Saadi", "Lounis", "Bouziane", "Medkour", "Gahlam"]

for index, row in teachers.iterrows():
    if any(teacher_name in row['teacher_name'] for teacher_name in groups_teachers):
        teacher = Teacher(row['teacher_id'], row['teacher_name'], row['field'], 1)
        teachers_3_year.append(teacher)
        
mappings = {
    'Sunday':1, 'Wednesday':4, 'Tuesday':3, 'Thursday':5, 'Monday':2
}
for index,row in working_day.iterrows():
    for teacher in teachers_3_year : 
        if row['teacher_id'] == teacher.id:
            teacher.prefered_days.append(mappings[row['week_day']])
    


In [9]:
for i in session_3_year:
    #print(i.subject,i.session_type)
    continue

In [10]:
# Double the sessions for each group

In [11]:
## adding some missing values from the database 
session_3_year[1].section = 2
session_3_year[6].section = 2
session_3_year.append(copy.deepcopy(session_3_year[8]))
session_3_year[-1].section = 2 
session_3_year.append(copy.deepcopy(session_3_year[11]))
session_3_year[-1].section = 2 
session_3_year.append(copy.deepcopy(session_3_year[13]))
session_3_year[-1].section = 2
session_3_year.append(copy.deepcopy(session_3_year[17]))
session_3_year[-1].section = 2 

In [12]:
##solving error in Dtabase 
for i in range(len(session_3_year)):
    if session_3_year[i].subject == "Project Management" and  session_3_year[i].session_type == "lab":
         session_3_year[i].session_type = "tuto"

In [13]:
for i in session_3_year:
    #print(i.subject,i.session_type)
    continue

In [14]:
# adding missing teachers 
teachers_3_year.append(Teacher("teacher20", "Rahma Djiroun", "Databases" , 1))
teachers_3_year.append(Teacher("teacher0", "Hadje Ameur", "Computer Science" , 1))
teachers_3_year.append(Teacher("teacher001", "Boukhalfa", "Databases" , 1))
teachers_3_year.append(Teacher("teacher0", "Gahlam Nadia", "Computer Science" , 1))

In [15]:
for i in teachers_3_year:
    print(i.name)

Aicha Boutorh
Seif Eddine Bouziane
Mohammed Brahimi
Houssem Brairi
Ahmed Guessoum
Meriem Amel Guessoum
Soumaya Lakehal
Noureddine Lasla
Karim Lounis
Ouarda Lounis
Tarek Medkour
Hayet Saadi
Rahma Djiroun
Hadje Ameur
Boukhalfa
Gahlam Nadia


In [16]:
teachers_3_year[10].preferef_period = "m"

In [17]:
teachers_3_year[1].preferef_period = "m"

In [18]:
teachers_3_year[12].preferef_period = "f"

In [19]:
##this needs ro be in the databse 
teacher_groups = {
    1: {
        "Advanced Databases tuto": "Meriem Amel Guessoum",
        "Advanced Databases lab": "Hayet Saadi",
        "Machine Learning tuto": "Hadje Ameur",
        "Machine Learning lab": "Aicha Boutorh",
        "Computer and network Security tuto": "Ouarda Lounis",
        "Computer and network Security lab": "Noureddine Lasla",
        "Time Series Analysis and Classification lab": "Houssem Brairi",
        "Numerical Methods and Optimisation lab": "Soumaya Lakehal",
        'Project Management tuto': "Gahlam Nadia"
    },
    2: {
        "Advanced Databases tuto": "Rahma Djiroun",
        "Advanced Databases lab": "Hayet Saadi",
        "Machine Learning tuto": "Hadje Ameur",
        "Machine Learning lab": "Seif Eddine Bouziane",
        "Computer and network Security tuto": "Ouarda Lounis",
        "Computer and network Security lab": "Ouarda Lounis",
        "Time Series Analysis and Classification lab": "Houssem Brairi",
        "Numerical Methods and Optimisation lab": "Soumaya Lakehal",
        'Project Management tuto': "Gahlam Nadia"
    },
    3: {
        "Advanced Databases tuto": "Rahma Djiroun",
        "Advanced Databases lab": "Hayet Saadi",
        "Machine Learning tuto": "Hadje Ameur",
        "Machine Learning lab": "Aicha Boutorh",
        "Computer and network Security tuto": "Ouarda Lounis",
        "Computer and network Security lab":  "Ouarda Lounis",
        "Time Series Analysis and Classification lab": "Houssem Brairi",
        "Numerical Methods and Optimisation lab": "Soumaya Lakehal",
        'Project Management tuto': "Gahlam Nadia"
    },
    4: {
        "Advanced Databases tuto": "Meriem Amel Guessoum",
        "Advanced Databases lab": "Hayet Saadi",
        "Machine Learning tuto": "Mohammed Brahimi",
        "Machine Learning lab": "Aicha Boutorh",
        "Computer and network Security tuto": "Ouarda Lounis",
        "Computer and network Security lab": "Noureddine Lasla",
        "Time Series Analysis and Classification lab": "Houssem Brairi",
        "Numerical Methods and Optimisation lab": "Soumaya Lakehal",
        'Project Management tuto': "Gahlam Nadia"
    },
    5: {
        "Advanced Databases tuto": "Rahma Djiroun",
        "Advanced Databases lab": "Hayet Saadi",
        "Machine Learning tuto": "Mohammed Brahimi",
        "Machine Learning lab": "Aicha Boutorh",
        "Computer and network Security tuto": "Ouarda Lounis",
        "Computer and network Security lab": "Noureddine Lasla",
        "Time Series Analysis and Classification lab": "Tarek Medkour",
        "Numerical Methods and Optimisation lab": "Soumaya Lakehal",
        'Project Management tuto': "Gahlam Nadia"

    },
    6: {
        "Advanced Databases tuto": "Meriem Amel Guessoum",
        "Advanced Databases lab": "Hayet Saadi",
        "Machine Learning tuto": "Mohammed Brahimi",
        "Machine Learning lab": "Seif Eddine Bouziane",
        "Computer and network Security tuto": "Ouarda Lounis",
        "Computer and network Security lab": "Noureddine Lasla",
        "Time Series Analysis and Classification lab": "Houssem Brairi",
        "Numerical Methods and Optimisation lab": "Soumaya Lakehal",
        'Project Management tuto': "Gahlam Nadia"

    }
}

section_group_dict = {
    1 : {1,2,3},
    2 : {4,5,6}
}

group_section_dict = {value: key for key, values in section_group_dict.items() for value in values}



In [20]:
sessions = []
for session in session_3_year:
    if session.session_type != "lecture" and session.subject!='Group-Project':
        for i in range(6):
            prof = None 
            for j in teachers_3_year : 
                if j.name == teacher_groups[i+1][f"{session.subject} {session.session_type}"]  : 
                    prof = j 
                    break
            duplicated_session = Session(session.session_type, prof , session.length, session.subject,group = i+1,section=group_section_dict[i+1])
            sessions.append(duplicated_session)

In [21]:

            
sessions.append(Session("lecture", teachers_3_year[8] ,1, "Computer and network Security",section=1))
sessions.append(Session("lecture", teachers_3_year[8] ,1, "Computer and network Security",section=2))

sessions.append(Session("lecture", teachers_3_year[-1] ,1, "Project Management",section=2))
sessions.append(Session("lecture", teachers_3_year[-1] ,1, "Project Management",section=1))

sessions.append(Session("lecture", teachers_3_year[2] ,1, "Machine Learning",section=2))
sessions.append(Session("lecture", teachers_3_year[4] ,1, "Machine Learning",section=1))

sessions.append(Session("lecture", teachers_3_year[-2] ,1, "Advanced Databases",section=2))
sessions.append(Session("lecture", teachers_3_year[-2] ,1, "Advanced Databases",section=1))

sessions.append(Session("lecture", teachers_3_year[10] ,1, "Time Series Analysis and Classification",section=2))
sessions.append(Session("lecture", teachers_3_year[10] ,1, "Time Series Analysis and Classification",section=1))



In [22]:
### now using sessions generate a timetable 
for i in sessions : 
    #print(i.group,i.subject,i.teacher.name,i.session_type)
    continue 

In [23]:
def mergeSort(arr,f_score,way="asc"):
    if len(arr) > 1:
        mid_indx = len(arr)//2
        left = arr[:mid_indx]
        right = arr[mid_indx:]
        f_left = f_score[:mid_indx]
        f_right = f_score[mid_indx:]
        mergeSort(left,f_left,way)
        mergeSort(right,f_right,way)
        i = j = k = 0
        if way == "dsc":
            while i < len(left) and j < len(right):
                if f_left[i] <= f_right[j]:
                    f_score[k] = f_left[i]
                    arr[k] = left[i]
                    i += 1
                else:
                    f_score[k] = f_right[j]
                    arr[k] = right[j]
                    j += 1
                k += 1
        elif  way == "asc":
            while i < len(left) and j < len(right):
                if f_left[i] >= f_right[j]:
                    f_score[k] = f_left[i]
                    arr[k] = left[i]
                    i += 1
                else:
                    f_score[k] = f_right[j]
                    arr[k] = right[j]
                    j += 1
                k += 1
 
        while i < len(left):
            f_score[k] = f_left[i]
            arr[k] = left[i]
            i += 1
            k += 1
 
        while j < len(right):
            f_score[k] = f_right[j]
            arr[k] = right[j]
            j += 1
            k += 1
    return arr,f_score

In [24]:
section_group_dict

{1: {1, 2, 3}, 2: {4, 5, 6}}

In [44]:
class Generator:
    
    def __init__(self,sessions,teachers,rooms,n_sections,n_groups,working_days,max_morning_periods,max_afternoon_periods,max_periods_day,section_group_dict,perecent=50):
        self.sessions = sessions
        # order by priority 
        self.teachers = sorted(teachers, key=lambda x: x.priority)
        self.rooms = rooms
        self.n_groups = n_groups
        self.n_sections = n_sections
        self.working_days = working_days  
        self.rests = ["f"] * (n_groups//(100//perecent) )
        self.rests += ["m"] * (n_groups//(100//(100-perecent))) # extend for more then 2 sessions : perecent is a list 
        random.shuffle(self.rests)
        self.max_morning_periods = max_morning_periods
        self.max_afternoon_periods = max_afternoon_periods
        self.max_periods_day = max_periods_day
        self.section_group_dict = section_group_dict
        
    def generate_combination_brute(self):
        # now shuffle and stack to generate 1000 possible child 
        sessions = copy.deepcopy(self.sessions)
        random.shuffle(sessions)
        possibilities = []
        
        while len(sessions2) > 0:
            period = []
            visited_groups = []
            visited_teachers = []
            for session in sessions[:]:  
                if len(period) == 6:
                    break
                if session.group not in visited_groups and session.teacher not in visited_teachers:
                    period.append(session)
                    visited_teachers.append(session.teacher)
                    visited_groups.append(session.group)
                    sessions.remove(session)  
            
            random.shuffle(period)
            possibilities.append(period)
        return possibilities

    def generate_combination(self):
        #for each groups, gather its session { '1' : .. , '2': ....sessions of g2 , ... 
        group_combinations = {}
        for i in range (self.n_groups):
            group_combinations[i+1] = []
        for i in self.sessions:
            if i.session_type == "lecture":
                for g in self.section_group_dict[i.section]:
                    group_combinations[g] += [i]
                    break #one group is enought as its gonna get generelized later on 
            else:
                group_combinations[i.group] += [i]
                random.shuffle(group_combinations[i.group])
            
        return group_combinations

    def populate(self,n):
        population = []
        remainder = []
        for i in range(n):
            group_combinations = self.generate_combination()
            result,remain = self.propagate(group_combinations)
            population.append(result)
            remainder.append(remain)
        return population,remainder
    
    def check_teacher(self,teacher, list,j):
        for i in list:
            teacher_obj = next((t for t in set(self.teachers) if i.teacher and t.id == i.teacher.id), None)
            if teacher == i.teacher or (teacher_obj and not teacher_obj.available[j]):
                return False
        return True
    
    def check_room(self,room, list,j):
        for i in list:
            room_obj = next((t for t in set(self.rooms) if i.room and t.id == i.room.id), None)
            if room == i.room or (room_obj and not room_obj.available[j]):
                return False
        return True

    def check_group(self,group, list):
        for i in list:
            if group == i.group:
                return False
        return True

    def is_empty(self,group_combinations):
        for period in group_combinations.values():
            if period:
                return False
        return True
        
    def check_lecture(self,choice,period):
        for i in period :
            if i.session_type == "lecture" :
                if choice.group in self.section_group_dict[i.section] or choice.group == i.group or choice.section == i.section:
                    return False
            if choice.session_type == "lecture" and i.group in self.section_group_dict[choice.section] :
                return False
        return True

    def check_lecture_2(self,period):
        n_periods =  0 # must sum up to  n_sections
        n_lectures = 0
        for i in period :
            if i.session_type=="lecture" :
                n_periods += len(self.section_group_dict[i.section])
                n_lectures+=1
            else :
                n_periods +=1
            if n_lectures > self.n_sections:
                return False
            if n_periods >= self.n_groups :
                    return False 
        return True

                
    def propagate(self,group_combinations, cool_down=10,max_itt = 1000):
        sl = []
        counter = 0
        while not self.is_empty(group_combinations) and max_itt>0 :
            starter = random.choice(list(group_combinations.keys()))
            if len(group_combinations[starter]) == 0:
                del group_combinations[starter]
                continue  # move to the next iteration if the selected group is empty
            period = [random.choice(group_combinations[starter])]
            group_combinations[starter].remove(period[0])
            if len(group_combinations[starter]) == 0:
                    del group_combinations[starter]
                    continue
            left_groups = [i for i in list(group_combinations.keys()) if i != starter]
            for i in left_groups:
                if len(group_combinations[i]) > 0:
                    choice = random.choice(group_combinations[i])
                    if self.check_teacher(choice.teacher, period,counter) and self.check_lecture(choice,period) and  self.check_lecture_2(period):
                        period.append(choice)
                        group_combinations[i].remove(choice)
                        if  self.check_lecture_2(period): #is full
                            continue
                        if len(group_combinations[i]) == 0 :
                                del group_combinations[i]
                                continue
                    try:
                        while not self.check_group(choice.group, period) and cool_down > 0:
                            cool_down -= 1
                            choice = random.choice(group_combinations[i])
                            if self.check_group(choice.group, period) and self.check_teacher(choice.teacher, period,counter) and self.check_lecture(choice,period) and  self.check_lecture_2(period):
                                period.append(choice)
                                group_combinations[i].remove(choice)
                                if self.check_lecture_2(period): #is full
                                    break
                                if len(group_combinations[i]) == 0:
                                    del group_combinations[i]
                    except:
                        pass
                    cool_down = 10
            sl.append(period)
            counter+=1
            if len(group_combinations[starter]) == 0:
                del group_combinations[starter]
            max_itt-=1
        return sl,group_combinations

    def is_valid(self,a,b,x,y):
        # class teacher module
        for index,i in enumerate(a) : 
            if b[y].group == i.group or b[y].teacher == i.teacher :
                return False
        for index,i in enumerate(b) : 
            if  a[x].group == i.group or a[x].teacher == i.teacher :
                return False
    
        return True
        
    def modify(self,time_table):
        max_iterations = 10000
        
        for _ in range(max_iterations):
            try:
                x = random.randint(0, len(time_table) - 1)
                y = random.randint(0, len(time_table[x]) - 1)
                u = random.randint(0, len(time_table) - 1)
                v = random.randint(0, len(time_table[u]) - 1)
            except:
                return time_table
            if  self.can_add_session(time_table[u][v],time_table[x],u) and self.can_add_session(time_table[x][y],time_table[u],x):
                time_table[x][y], time_table[u][v] = time_table[u][v], time_table[x][y]
                return time_table
    
        return time_table

    def rearange(self,time_table, max_itt,max_stuck ):
        
        best = time_table
        current = time_table
        best_score = self.heuristic(time_table)
        itt = max_itt
        stuck = max_stuck
        
        while itt > 0 :
            new = self.modify(copy.deepcopy(current))
            if self.heuristic(new) > best_score :
                best = new
                best_score = self.heuristic(new)
            if False and self.heuristic(new) == best_score and new != best:
                # new branch 
                posibility = rearange(copy.deepcopy(new),heuristic,copy.deepcopy(itt),copy.deepcopy(stuck))
                if self.heuristic(posibility) > best_score :
                    best = posibility
                    best_score = self.heuristic(posibility)
            if self.heuristic(new) >= self.heuristic(current) :
                current = new
            else :
                stuck-=1 
            if stuck == 0 :
                # allow a drop 
                stuck = max_stuck
                current = new
            itt -= 1
            
        return best

    def can_add_session(self, session, period,i):
        return self.check_group(session.group, period) and self.check_teacher(session.teacher, period,i) and self.check_lecture(session,period) and self.check_lecture_2(period)

    
    def search_2(self, time_table, max_itt=10, max_itt_2=100,r=None,rem=None):
        if r and rem :
            result = r
            remaining = rem
        else : 
            result = [[] for _ in range(self.max_periods_day * len(self.working_days))]
            remaining = [session for period in time_table for session in period]
            
        while len(remaining) > 0 and max_itt_2 > 0:
            temp = max_itt
            for i in range(len(result)):
                while self.check_lecture_2(result[i]) and max_itt > 0:
                    best_posibility = None
                    best_h = -1
                    for session in remaining:
                        if self.can_add_session(session, result[i],i):
                            p = copy.deepcopy(result)
                            p[i].append(session)
                            h = self.heuristic(p)
                            if h > best_h:
                                best_posibility = session
                                best_h = h
                    if best_posibility:
                        result[i].append(best_posibility)
                        remaining.remove(best_posibility)
                    max_itt -= 1
                    if max_itt == 0:
                        max_itt = temp
                        break  # Exit the inner loop if max_itt is reached
    
            max_itt_2 -= 1
    
        return result,remaining


    def search_3(self):
        # start fulifying constraints one by one, ex teachre with highest priority - > preference ... keep a list of constraints that did not get fulified         
        time_table =  [[] for _ in range(self.max_periods_day * len(self.working_days))]#self.init_time_table()
        sessions = copy.deepcopy(self.sessions)
        visited = []
        teachers_sessions  = {}
        for session in sessions :
            if  session.teacher not in teachers_sessions :
                teachers_sessions[session.teacher] = [session]
            else : 
                teachers_sessions[session.teacher].append(session)
        
        for teacher in teachers_sessions.keys(): # they are ordered by priority ( check init function )
            if teacher.prefered_days :
                for d in teacher.prefered_days:
                    d-=1
                    for session in teachers_sessions[teacher] :
                        if  session not in visited : 
                            if teacher.preferef_period == "f" : #likes morning 
                                    counter = 0
                                    time = d*(self.max_morning_periods+self.max_afternoon_periods)
                                    current_hour = i%(self.max_periods_day)
                                    mh = self.max_morning_periods - (1 if session.group and  self.rests[session.group-1] == "m" else 0)
                                    ah = self.max_afternoon_periods - (1 if  session.group and self.rests[session.group-1] == "f" else 0)
                                    while counter+mh+time<len(time_table) and counter<ah:
                                        if self.can_add_session(session, time_table[time+counter+mh],time+counter+mh):
                                            time_table[time+counter+mh].append(session)
                                            visited.append(session)
                                            break
                                        counter+=1
                            elif teacher.preferef_period == "m": 
                                    counter =  0
                                    time = d*(self.max_morning_periods+self.max_afternoon_periods)
                                    current_hour = i%(self.max_periods_day)
                                    mh = self.max_morning_periods - (1 if  session.group and self.rests[session.group-1] == "m" else 0)
                                    ah = self.max_afternoon_periods - (1 if session.group and  self.rests[session.group-1] == "f" else 0)
                                    while counter+time<len(time_table) and counter<mh:
                                        if self.can_add_session(session, time_table[time+counter],time+counter):
                                            time_table[time+counter].append(session)
                                            visited.append(session)
                                            break
                                        counter+=1
                            else :
                                counter =  0
                                time = d*(self.max_morning_periods+self.max_afternoon_periods)
                                while counter+time<len(time_table) and counter<self.max_morning_periods+self.max_afternoon_periods:
                                    if self.can_add_session(session, time_table[time+counter],time+counter):
                                        time_table[time+counter].append(session)
                                        visited.append(session)
                                        break
                                    counter+=1
            #####################"
            if teacher.preferef_period :
                # make this teacher's session at the period he likes if possible, else leave them till the end 
                for session in teachers_sessions[teacher] : 
                    if  session not in visited :
                        if teacher.preferef_period == "m" : #likes morning 
                            for time in range(0,len(time_table),self.max_morning_periods+self.max_afternoon_periods):
                                counter =  0
                                mh = self.max_morning_periods - (1 if session.group and self.rests[session.group-1] == "m" else 0)
                                ah = self.max_afternoon_periods - (1 if session.group and  self.rests[session.group-1] == "f" else 0)
                                while counter+time<len(time_table) and counter<mh:
                                    if self.can_add_session(session, time_table[time+counter],time+counter):
                                        time_table[time+counter].append(session)
                                        visited.append(session)
                                        break
                                    counter+=1
                                
                        elif  teacher.preferef_period == "f": 
                            for time in range(0,len(time_table),self.max_morning_periods+self.max_afternoon_periods):
                                counter = 0
                                mh = self.max_morning_periods - (1 if session.group and self.rests[session.group-1] == "m" else 0)
                                ah = self.max_afternoon_periods - (1 if session.group and  self.rests[session.group-1] == "f" else 0)
                                while counter+mh+time<len(time_table) and counter<ah:
                                    if self.can_add_session(session, time_table[time+counter+mh],time+counter+mh):
                                        time_table[time+counter+mh].append(session)
                                        visited.append(session)
                                        break
                                    counter+=1
        sessions = [session for session in sessions if session not in visited]
        if len(sessions)>0:
            return self.search_2(time_table, max_itt=10, max_itt_2=100,r=time_table,rem=sessions)
            
        return  time_table,sessions                 
                        
    def search(self,population, max_itt,max_stuck):
        soluion = []
        score = []
        for i in population :
            result = self.rearange(copy.deepcopy(i), max_itt,max_stuck)
            soluion.append(result)
            score.append(self.heuristic(result))
            soluion,score =  mergeSort(soluion,score,"asc")
    
        return soluion,score 
    
    def neighbor_function(self,current_solution,max_itt,max_stuck):
        return self.rearange(current_solution,max_itt,max_stuck)


    def simulated_annealing(self,initial_solution, max_temp,min_temp,max_itt,max_stuck,cooling_schedule=0.01):
        
        current_solution = initial_solution
        current_temperature = max_temp
        energy = self.heuristic(current_solution)
        while current_temperature > min_temp:  
            neighbor_solution = self.neighbor_function(current_solution,max_itt,max_stuck)
            delta_heuristic = self.heuristic(neighbor_solution) - energy
            if delta_heuristic > 0 or random.random() < self.acceptance_probability(delta_heuristic, current_temperature):
                current_solution = neighbor_solution   
                energy = delta_heuristic
            current_temperature *= cooling_schedule
            
        return current_solution

    def acceptance_probability(self,delta, temperature):
        if delta > 0:
            return 1.0
        else:
            return np.exp(-delta / temperature)

    def heuristic(self,time_table):
        #self.priority = priority
        #self.preferef_period = preferef_period # m : morning, f:afternoon, None:anything is fine
        result = 0
        for i,period in enumerate(time_table):
            for teacher in self.teachers:
                if teacher.prefered_days: 
                    for d in teacher.prefered_days : 
                        time = d*(self.max_periods_day)
                        if i >= time and i <= time + self.max_periods_day:
                            result += 3
                if teacher.preferef_period : 
                        for session in period:
                            if session.teacher == teacher and session.group:
                                current_hour = i%(self.max_periods_day)
                                mh = self.max_morning_periods
                                ah = self.max_afternoon_periods
                                if self.rests[session.group-1] == "m":
                                    mh-=1
                                else :
                                    ah-=1
                                if (teacher.preferef_period == "m" and current_hour < mh) or (teacher.preferef_period == "f" and current_hour >= mh and current_hour < mh+ah):
                                    result += 2  # costraint teacher prefer working at morning/afternoon is met 

            ### emphasize stacking groups together can be removes 
            if len(period)>=5 : 
                result += 1
        
        return result

    def init_time_table(self):
        time_table = []
        for i in range(len(self.working_days)*(self.max_morning_periods+self.max_afternoon_periods)):
            time_table.append([])
        # randomly fill with epmty sessions befor it starts 
        for group_id in range(self.n_groups):
            for time in range(0,len(time_table),self.max_morning_periods+self.max_afternoon_periods):
                range_a = 0 if self.rests[group_id] == "m" else self.max_morning_periods
                range_b = self.max_morning_periods if self.rests[group_id] == "m" else self.max_morning_periods+self.max_afternoon_periods
                counter = random.randint(range_a,range_b)
                cool = 1000
                while time+counter>=len(time_table) and cool>0:
                    range_a = 0 if self.rests[group_id] == "m" else self.max_morning_periods
                    range_b = self.max_morning_periods if self.rests[group_id] == "m" else self.max_morning_periods+self.max_afternoon_periods
                    counter = random.randint(range_a,range_b)
                    cool-=1
                if cool == 0 and time+counter>=len(time_table):
                    counter = random.randint(time,len(time_table)-1)
                    time_table[counter].append(Session("rest", None , 1, None ,group = group_id+1,section=group_section_dict[group_id+1]))
                    continue
                time_table[time+counter].append(Session("rest", None , 1, None ,group = group_id+1,section=group_section_dict[group_id+1]))
                
        return time_table
    
    def assign_rooms(self,schedule):
        remainder = []
        for period_idx, period in enumerate(schedule):
            time = period_idx % (self.max_afternoon_periods + self.max_morning_periods)
            empty_rooms = [copy.deepcopy(r) for r in self.rooms if  r.available[time] ]
            for session in schedule[period_idx]:
                if session.session_type != "rest":
                    group = session.group
                    room = next((room for room in empty_rooms if room.room_type == session.session_type ), None)
                    if room:
                        # assigne the room to the the session
                        schedule[period_idx][schedule[period_idx].index(session)].room = room 
                        empty_rooms.remove(room) 
        for period_idx, period in enumerate(schedule):
            for session in schedule[period_idx]:
                if not session.room:
                    remainder.append(session)
        return schedule,session
    
    def reserve_teacher(self,timetable):
        
        temp = set(self.teachers)
        for i, period in enumerate(timetable):
            for session in period:
                teacher_obj = next((teacher for teacher in temp if session.teacher and teacher.id == session.teacher.id), None)
                if teacher_obj:
                    teacher_obj.available[i % (self.max_afternoon_periods + self.max_morning_periods)] = False
        self.teachers = list(temp)
    
    def reserve_rooms(self,timetable):
        temp = set(self.rooms)
        for i, period in enumerate(timetable):
            for session in period:
                room_obj = next((room for room in temp if session.room and room.id == session.room.id), None)
                if room_obj:
                    room_obj.available[i % (self.max_afternoon_periods + self.max_morning_periods)] = False
        self.rooms = list(temp)
        
    def group_has_lecture(self,group,period):
        for i in period:
            if i.session_type == "lecture" :
                if group in self.section_group_dict[i.section]:
                    return True
        return False
        
    def add_rests(self, timetable):
        reschedule = []
        for time in range(0, len(timetable), self.max_morning_periods + self.max_afternoon_periods):
            for group_id, i in enumerate(self.rests): 
                if i == "m":
                    for j in reversed(range(0, self.max_morning_periods)):
                        f = False
                       
                        if time + j < len(timetable) and not self.group_has_lecture(group_id+1,timetable[time + j]) :
                            groyp_exists = False 
                            for session in timetable[time + j]:
                                if session.group == group_id + 1:
                                    groyp_exists = True
                                if session.session_type !="lecture" and session.group == group_id + 1:
                                    reschedule.append(session)
                                    timetable[time + j].remove(session)
                                    timetable[time + j].append(Session("rest", None, 1, None, group=session.group, section=session.section))
                                    f = True
                                    break
                            if not groyp_exists:
                                timetable[time + j].append(Session("rest", None, 1, None, group=session.group, section=session.section))
                                f = True
                                break
                        if f :
                            break
                        
                elif i == "f":
                    for j in reversed(range(self.max_morning_periods, self.max_morning_periods+self.max_afternoon_periods)):
                        f = False
                        if time + j < len(timetable) and not self.group_has_lecture(group_id+1,timetable[time + j]):
                            groyp_exists = False 
                            for session in timetable[time + j]:
                                if session.session_type !="lecture" and session.group == group_id + 1 :
                                    reschedule.append(session)
                                    timetable[time + j].remove(session)
                                    timetable[time + j].append(Session("rest", None, 1, None, group=session.group, section=session.section))
                                    f = True
                                    break
                            if not groyp_exists:
                                timetable[time + j].append(Session("rest", None, 1, None, group=session.group, section=session.section))
                                f = True
                                break
                        if f :
                            break
        return self.search_2(timetable, max_itt=10, max_itt_2=100, r=timetable, rem=reschedule)


In [45]:
generator = Generator(sessions,teachers_3_year,available_rooms,2,6,working_days,max_morning_periods,max_afternoon_periods,max_periods_day,section_group_dict)
print(generator.rests)


['f', 'm', 'm', 'f', 'f', 'm']


In [27]:
group_conmbinations = generator.populate(15)[0]

In [28]:
timetables,scores = generator.search(group_conmbinations, 1000 , 100 )

In [29]:
for i in range(len(timetables)) : 
    timetables[i] = generator.assign_rooms(timetables[i])[0]

In [30]:
def print_schedule(schedule):
    for i, period in enumerate(schedule):
        print(f"Period {i+1}:")
        for session in period:
            try:
                print(f"{session.session_type} , - Group: {session.group} - Teacher: {session.teacher.name}, Subject: {session.subject}, Room: {session.room.id}")
            except:
                print(f"{session.session_type}")
#print_schedule(timetables[0]) 

In [31]:
print("best score ",scores[0])

best score  9425


In [32]:
import pandas as pd

columns = ['Group 1', 'Group 2', 'Group 3', 'Group 4', 'Group 5', 'Group 6']
index = pd.MultiIndex.from_product([range(1, 6), ['Morning 1', 'Morning 2', 'Morning 3' , 'Afternoon 1', 'Afternoon 2','Afternoon 3']], names=['Day', 'Slot'])
df = pd.DataFrame(columns=columns, index=index)
i = 0

for period in timetables[0]:
        for j, s in enumerate(period):
            try:
                teacher_subject_room = s.teacher.name + " " + s.subject + ' ' + s.room.id
                #print(teacher_subject_room)
                if s.session_type == "lecture":
                    for k in section_group_dict[s.section]:
                        df.iloc[i][f'Group {k}'] = teacher_subject_room
                else:
                    df.iloc[i][f'Group {s.group}'] = teacher_subject_room
            except:
                df.iloc[i][f'Group {s.group}'] = "REST"
        i += 1


In [33]:
df

Group 1  \
Day Slot                                                             
1   Morning 1    Houssem Brairi Time Series Analysis and Classi...   
    Morning 2                                                  NaN   
    Morning 3                  Aicha Boutorh Machine Learning lab1   
    Afternoon 1                Aicha Boutorh Machine Learning lab1   
    Afternoon 2                 Hadje Ameur Machine Learning tuto2   
    Afternoon 3  Karim Lounis Computer and network Security amphi1   
2   Morning 1        Meriem Amel Guessoum Advanced Databases tuto1   
    Morning 2    Ouarda Lounis Computer and network Security tuto4   
    Morning 3               Ahmed Guessoum Machine Learning amphi1   
    Afternoon 1  Noureddine Lasla Computer and network Security...   
    Afternoon 2                                                NaN   
    Afternoon 3  Soumaya Lakehal Numerical Methods and Optimisa...   
3   Morning 1                  Boukhalfa Advanced Databases amphi1   
    Morning 2                Gahlam Nadia Project Management tuto1   
    Morning 3               Gahlam Nadia Project Management amphi1   
    Afternoon 1                Hayet Saadi Advanced Databases lab2   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   
4   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   
5   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   

                                                           Group 2  \
Day Slot                                                             
1   Morning 1    Soumaya Lakehal Numerical Methods and Optimisa...   
    Morning 2           Seif Eddine Bouziane Machine Learning lab1   
    Morning 3     Ouarda Lounis Computer and network Security lab2   
    Afternoon 1  Houssem Brairi Time Series Analysis and Classi...   
    Afternoon 2                                                NaN   
    Afternoon 3  Karim Lounis Computer and network Security amphi1   
2   Morning 1                  Hayet Saadi Advanced Databases lab2   
    Morning 2                   Hadje Ameur Machine Learning tuto1   
    Morning 3               Ahmed Guessoum Machine Learning amphi1   
    Afternoon 1  Ouarda Lounis Computer and network Security tuto2   
    Afternoon 2              Gahlam Nadia Project Management tuto1   
    Afternoon 3                                                NaN   
3   Morning 1                  Boukhalfa Advanced Databases amphi1   
    Morning 2               Rahma Djiroun Advanced Databases tuto2   
    Morning 3               Gahlam Nadia Project Management amphi1   
    Afternoon 1         Seif Eddine Bouziane Machine Learning lab1   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   
4   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN  

In [34]:
result_s2 = generator.assign_rooms(generator.search_2(group_conmbinations[0])[0])[0]

import pandas as pd

columns = ['Group 1', 'Group 2', 'Group 3', 'Group 4', 'Group 5', 'Group 6']
index = pd.MultiIndex.from_product([range(1, 6), ['Morning 1', 'Morning 2', 'Morning 3', 'Afternoon 1', 'Afternoon 2','Afternoon 3']], names=['Day', 'Slot'])
df = pd.DataFrame(columns=columns, index=index)
i = 0

for period in result_s2:
        for j, session in enumerate(period):
            if session.session_type != "rest":
                if session.room:
                    teacher_subject_room = session.teacher.name + " " + session.subject + ' ' + session.room.id
                else : 
                    teacher_subject_room = session.teacher.name + " " + session.subject + ' ' + " NO ROOM "
                if session.session_type == "lecture":
                    for k in section_group_dict[session.section]:
                        df.iloc[i][f'Group {k}'] = teacher_subject_room
                else :
                    df.iloc[i][f'Group {session.group}'] = teacher_subject_room
            else : 
                df.iloc[i][f'Group {session.group}'] = "REST"
        i+=1 

In [35]:
df

Group 1  \
Day Slot                                                             
1   Morning 1                   Hadje Ameur Machine Learning tuto1   
    Morning 2                                                  NaN   
    Morning 3                  Aicha Boutorh Machine Learning lab3   
    Afternoon 1                                                NaN   
    Afternoon 2                Aicha Boutorh Machine Learning lab2   
    Afternoon 3  Ouarda Lounis Computer and network Security tuto1   
2   Morning 1    Soumaya Lakehal Numerical Methods and Optimisa...   
    Morning 2    Karim Lounis Computer and network Security amphi1   
    Morning 3    Noureddine Lasla Computer and network Security...   
    Afternoon 1      Meriem Amel Guessoum Advanced Databases tuto1   
    Afternoon 2             Gahlam Nadia Project Management amphi1   
    Afternoon 3  Houssem Brairi Time Series Analysis and Classi...   
3   Morning 1               Ahmed Guessoum Machine Learning amphi1   
    Morning 2                  Hayet Saadi Advanced Databases lab2   
    Morning 3                Gahlam Nadia Project Management tuto1   
    Afternoon 1  Tarek Medkour Time Series Analysis and Classif...   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   
4   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   
5   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   

                                                           Group 2  \
Day Slot                                                             
1   Morning 1           Seif Eddine Bouziane Machine Learning lab1   
    Morning 2                   Hadje Ameur Machine Learning tuto2   
    Morning 3           Seif Eddine Bouziane Machine Learning lab1   
    Afternoon 1             Rahma Djiroun Advanced Databases tuto1   
    Afternoon 2              Gahlam Nadia Project Management tuto2   
    Afternoon 3  Soumaya Lakehal Numerical Methods and Optimisa...   
2   Morning 1     Ouarda Lounis Computer and network Security lab2   
    Morning 2    Karim Lounis Computer and network Security amphi1   
    Morning 3    Houssem Brairi Time Series Analysis and Classi...   
    Afternoon 1                                                NaN   
    Afternoon 2             Gahlam Nadia Project Management amphi1   
    Afternoon 3                Hayet Saadi Advanced Databases lab1   
3   Morning 1               Ahmed Guessoum Machine Learning amphi1   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1  Tarek Medkour Time Series Analysis and Classif...   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   
4   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN  

In [46]:
result_s3 = generator.assign_rooms(generator.search_3()[0])[0]

import pandas as pd

columns = ['Group 1', 'Group 2', 'Group 3', 'Group 4', 'Group 5', 'Group 6']
index = pd.MultiIndex.from_product([range(1, 6), ['Morning 1', 'Morning 2', 'Morning 3', 'Afternoon 1', 'Afternoon 2','Afternoon 3']], names=['Day', 'Slot'])
df = pd.DataFrame(columns=columns, index=index)
i = 0

for period in result_s3:
        for j, session in enumerate(period):
            if session.session_type != "rest":
                if session.room:
                    teacher_subject_room = session.teacher.name + " " + session.subject + ' ' + session.room.id
                else : 
                    teacher_subject_room = session.teacher.name + " " + session.subject + ' ' + " NO ROOM "
                if session.session_type == "lecture":
                    for k in section_group_dict[session.section]:
                        df.iloc[i][f'Group {k}'] = teacher_subject_room
                else :
                    df.iloc[i][f'Group {session.group}'] = teacher_subject_room
            else : 
                df.iloc[i][f'Group {session.group}'] = "REST"
        i+=1 

In [47]:
df

Group 1  \
Day Slot                                                             
1   Morning 1                   Hadje Ameur Machine Learning tuto3   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1              Gahlam Nadia Project Management tuto2   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   
2   Morning 1                  Aicha Boutorh Machine Learning lab2   
    Morning 2    Soumaya Lakehal Numerical Methods and Optimisa...   
    Morning 3    Ouarda Lounis Computer and network Security tuto4   
    Afternoon 1  Noureddine Lasla Computer and network Security...   
    Afternoon 2             Gahlam Nadia Project Management amphi2   
    Afternoon 3  Houssem Brairi Time Series Analysis and Classi...   
3   Morning 1    Tarek Medkour Time Series Analysis and Classif...   
    Morning 2        Meriem Amel Guessoum Advanced Databases tuto4   
    Morning 3                  Hayet Saadi Advanced Databases lab8   
    Afternoon 1                                                NaN   
    Afternoon 2  Karim Lounis Computer and network Security amphi2   
    Afternoon 3             Ahmed Guessoum Machine Learning amphi2   
4   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                Boukhalfa Advanced Databases amphi2   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   
5   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   

                                                           Group 2  \
Day Slot                                                             
1   Morning 1                                                  NaN   
    Morning 2                   Hadje Ameur Machine Learning tuto3   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2              Gahlam Nadia Project Management tuto3   
    Afternoon 3                                                NaN   
2   Morning 1           Seif Eddine Bouziane Machine Learning lab4   
    Morning 2    Ouarda Lounis Computer and network Security tuto5   
    Morning 3    Soumaya Lakehal Numerical Methods and Optimisa...   
    Afternoon 1  Houssem Brairi Time Series Analysis and Classi...   
    Afternoon 2             Gahlam Nadia Project Management amphi2   
    Afternoon 3   Ouarda Lounis Computer and network Security lab5   
3   Morning 1    Tarek Medkour Time Series Analysis and Classif...   
    Morning 2                  Hayet Saadi Advanced Databases lab6   
    Morning 3               Rahma Djiroun Advanced Databases tuto4   
    Afternoon 1                                                NaN   
    Afternoon 2  Karim Lounis Computer and network Security amphi2   
    Afternoon 3             Ahmed Guessoum Machine Learning amphi2   
4   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                Boukhalfa Advanced Databases amphi2   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN  

In [48]:
res = generator.add_rests(result_s3)[0]

columns = ['Group 1', 'Group 2', 'Group 3', 'Group 4', 'Group 5', 'Group 6']
index = pd.MultiIndex.from_product([range(1, 6), ['Morning 1', 'Morning 2', 'Morning 3' , 'Afternoon 1', 'Afternoon 2','Afternoon 3']], names=['Day', 'Slot'])
df = pd.DataFrame(columns=columns, index=index)
i = 0
for period in res:
        for j, session in enumerate(period):
            if session.session_type != "rest":
                try:
                    teacher_subject_room = session.teacher.name + " " + session.subject + ' ' + session.room.id
                    if session.session_type == "lecture":
                        for k in section_group_dict[session.section]:
                            df.iloc[i][f'Group {k}'] = teacher_subject_room
                    else :
                        df.iloc[i][f'Group {session.group}'] = teacher_subject_room
                except :
                    df.iloc[i][f'Group {session.group}'] =   session.teacher.name + " " + session.subject
            else :
                df.iloc[i][f'Group {session.group}'] = "REST"
        i+=1 
df

Group 1  \
Day Slot                                                             
1   Morning 1                   Hadje Ameur Machine Learning tuto3   
    Morning 2                                                  NaN   
    Morning 3    Houssem Brairi Time Series Analysis and Classi...   
    Afternoon 1              Gahlam Nadia Project Management tuto2   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   
2   Morning 1                  Aicha Boutorh Machine Learning lab2   
    Morning 2    Soumaya Lakehal Numerical Methods and Optimisa...   
    Morning 3    Ouarda Lounis Computer and network Security tuto4   
    Afternoon 1  Noureddine Lasla Computer and network Security...   
    Afternoon 2             Gahlam Nadia Project Management amphi2   
    Afternoon 3                                               REST   
3   Morning 1    Tarek Medkour Time Series Analysis and Classif...   
    Morning 2        Meriem Amel Guessoum Advanced Databases tuto4   
    Morning 3                  Hayet Saadi Advanced Databases lab8   
    Afternoon 1                                                NaN   
    Afternoon 2  Karim Lounis Computer and network Security amphi2   
    Afternoon 3             Ahmed Guessoum Machine Learning amphi2   
4   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                Boukhalfa Advanced Databases amphi2   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   
5   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   

                                                           Group 2  \
Day Slot                                                             
1   Morning 1                                                  NaN   
    Morning 2                   Hadje Ameur Machine Learning tuto3   
    Morning 3               Rahma Djiroun Advanced Databases tuto4   
    Afternoon 1                                                NaN   
    Afternoon 2              Gahlam Nadia Project Management tuto3   
    Afternoon 3                                                NaN   
2   Morning 1           Seif Eddine Bouziane Machine Learning lab4   
    Morning 2    Ouarda Lounis Computer and network Security tuto5   
    Morning 3                                                 REST   
    Afternoon 1  Houssem Brairi Time Series Analysis and Classi...   
    Afternoon 2             Gahlam Nadia Project Management amphi2   
    Afternoon 3   Ouarda Lounis Computer and network Security lab5   
3   Morning 1    Tarek Medkour Time Series Analysis and Classif...   
    Morning 2                  Hayet Saadi Advanced Databases lab6   
    Morning 3                                                 REST   
    Afternoon 1  Soumaya Lakehal Numerical Methods and Optimisa...   
    Afternoon 2  Karim Lounis Computer and network Security amphi2   
    Afternoon 3             Ahmed Guessoum Machine Learning amphi2   
4   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                Boukhalfa Advanced Databases amphi2   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN  

In [49]:
generator.heuristic(res)

10573

In [40]:
result_s3 = generator.simulated_annealing(group_conmbinations[-1],2,0.1,100,100)


columns = ['Group 1', 'Group 2', 'Group 3', 'Group 4', 'Group 5', 'Group 6']
index = pd.MultiIndex.from_product([range(1, 6), ['Morning 1', 'Morning 2', 'Morning 3', 'Afternoon 1', 'Afternoon 2','Afternoon 3']], names=['Day', 'Slot'])
df = pd.DataFrame(columns=columns, index=index)
i = 0

for period in result_s3:
        for j, session in enumerate(period):
            if session.session_type != "rest":
                if session.room:
                    teacher_subject_room = session.teacher.name + " " + session.subject + ' ' + session.room.id
                else : 
                    teacher_subject_room = session.teacher.name + " " + session.subject + ' ' + " NO ROOM "
                if session.session_type == "lecture":
                    for k in section_group_dict[session.section]:
                        df.iloc[i][f'Group {k}'] = teacher_subject_room
                else :
                    df.iloc[i][f'Group {session.group}'] = teacher_subject_room
            else : 
                df.iloc[i][f'Group {session.group}'] = "REST"
        i+=1 
df

Group 1  \
Day Slot                                                             
1   Morning 1    Ouarda Lounis Computer and network Security tuto1   
    Morning 2    Houssem Brairi Time Series Analysis and Classi...   
    Morning 3    Karim Lounis Computer and network Security amphi1   
    Afternoon 1                Hayet Saadi Advanced Databases lab2   
    Afternoon 2      Meriem Amel Guessoum Advanced Databases tuto1   
    Afternoon 3             Gahlam Nadia Project Management amphi1   
2   Morning 1                Gahlam Nadia Project Management tuto1   
    Morning 2                                                  NaN   
    Morning 3    Noureddine Lasla Computer and network Security...   
    Afternoon 1                 Hadje Ameur Machine Learning tuto1   
    Afternoon 2             Boukhalfa Advanced Databases  NO ROOM    
    Afternoon 3                                                NaN   
3   Morning 1    Soumaya Lakehal Numerical Methods and Optimisa...   
    Morning 2                  Aicha Boutorh Machine Learning lab2   
    Morning 3    Tarek Medkour Time Series Analysis and Classif...   
    Afternoon 1             Ahmed Guessoum Machine Learning amphi1   
    Afternoon 2                Aicha Boutorh Machine Learning lab2   
    Afternoon 3                                                NaN   
4   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   
5   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN   

                                                           Group 2  \
Day Slot                                                             
1   Morning 1                  Hayet Saadi Advanced Databases lab1   
    Morning 2     Ouarda Lounis Computer and network Security lab2   
    Morning 3    Karim Lounis Computer and network Security amphi1   
    Afternoon 1                                                NaN   
    Afternoon 2             Rahma Djiroun Advanced Databases tuto1   
    Afternoon 3             Gahlam Nadia Project Management amphi1   
2   Morning 1           Seif Eddine Bouziane Machine Learning lab1   
    Morning 2           Seif Eddine Bouziane Machine Learning lab1   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2             Boukhalfa Advanced Databases  NO ROOM    
    Afternoon 3  Houssem Brairi Time Series Analysis and Classi...   
3   Morning 1                Gahlam Nadia Project Management tuto2   
    Morning 2                   Hadje Ameur Machine Learning tuto2   
    Morning 3    Tarek Medkour Time Series Analysis and Classif...   
    Afternoon 1             Ahmed Guessoum Machine Learning amphi1   
    Afternoon 2                                                NaN   
    Afternoon 3  Ouarda Lounis Computer and network Security  N...   
4   Morning 1                                                  NaN   
    Morning 2                                                  NaN   
    Morning 3                                                  NaN   
    Afternoon 1                                                NaN   
    Afternoon 2                                                NaN   
    Afternoon 3                                                NaN  

In [41]:
# after approving a time table, call generator.add_rests(timetables) set teachers spots to not available, the same for rooms, then call the function again to generator for another year ... 
# when you call for another year, you need to set the sessions variables again,  number of groups and sections  , then call one of the three searches 
generator.reserve_teacher(result_s3)
generator.reserve_rooms(result_s3)

In [42]:
for t in generator.teachers :
    print(t.name , t.available)

Ahmed Guessoum [True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
Soumaya Lakehal [False, True, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
Houssem Brairi [True, False, True, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
Hadje Ameur [False, False, False, False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
Hayet Saadi [False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
Rahma Djiroun [False, False, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 

In [43]:
for t in generator.rooms :
    print(t.id , t.available)

tuto1 [False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
tuto2 [False, False, False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
tuto3 [True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
tuto7 [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
tuto8 [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
tuto26 [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
lab8 [True, True, True, True, True, True, True